In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from scipy.stats import skew
%matplotlib inline

# Data load

In [2]:
train_df = pd.read_csv('../dat/train.csv')
print(train_df.shape)
train_df.head(3)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [3]:
train_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

# Missing Vals

In [4]:
# Number of colomns with nulls
def check_nulls(df):
    return len(df.isnull().sum().nonzero()[0])

In [5]:
print('Number of colomns with nulls: ', check_nulls(train_df))

Number of colomns with nulls:  19


## Object type fill in missing vals:

In [6]:
train_df_obj = train_df.select_dtypes(include=['object'])
print(train_df_obj.shape)


(1460, 43)


In [7]:
print('Number of colomns with nulls: ', check_nulls(train_df_obj))

Number of colomns with nulls:  16


## FILLING ALL with None is not always the good way. Sometimes there's a specific default value per colomns

In [8]:
train_df_obj = train_df_obj.fillna("None")

In [9]:
print('Number of colomns with nulls: ', check_nulls(train_df_obj))

Number of colomns with nulls:  0


## Numerical missing values handling:

In [10]:
imp = Imputer()
train_df_non_obj = train_df.select_dtypes(exclude=['object'])
print(train_df_non_obj.shape)

(1460, 38)


## You must drop the target colomn

In [11]:
train_df_non_obj = train_df_non_obj.drop('SalePrice', axis=1)
print(train_df_non_obj.shape)

(1460, 37)


In [12]:
print('Number of colomns with nulls: ', check_nulls(train_df_non_obj))

Number of colomns with nulls:  3


In [13]:
#train_df_non_obj = imp.fit_transform(train_df_non_obj)
train_df_non_obj = pd.DataFrame(imp.fit_transform(train_df_non_obj), index=train_df_non_obj.index, columns=train_df_non_obj.columns)


In [14]:
print('Number of colomns with nulls: ', check_nulls(train_df_non_obj))

Number of colomns with nulls:  0


## Encode the skewed cols into log transform

In [15]:

# Log transform of the skewed numerical features to lessen impact of outliers
# Inspired by Alexandru Papiu's script : https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models
# As a general rule of thumb, a skewness with an absolute value > 0.5 is considered at least moderately skewed
skewness = train_df_non_obj.apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.5]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
train_df_non_obj[skewed_features] = np.log1p(train_df_non_obj[skewed_features])

29 skewed numerical features to log transform


## Now put the two transfroms together

In [16]:
train_df = pd.concat([train_df_non_obj, train_df_obj, train_df.SalePrice], axis=1)# Dont forget to merge back the target price!

In [17]:
train_df.shape

(1460, 81)

In [18]:
print('Number of colomns with nulls: ', check_nulls(train_df))

Number of colomns with nulls:  0


# Let's find the features importance by correlation to price

In [19]:
# Find most important features relative to target
print("Find most important features relative to target")
corr = train_df.corr()
corr.sort_values(["SalePrice"], ascending = False, inplace = True)
print(corr.SalePrice)

Find most important features relative to target
SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.695147
GarageCars       0.640409
GarageArea       0.623431
1stFlrSF         0.591299
FullBath         0.560664
TotRmsAbvGrd     0.522355
YearBuilt        0.520575
YearRemodAdd     0.506848
Fireplaces       0.480136
GarageYrBlt      0.468325
OpenPorchSF      0.429970
MasVnrArea       0.411655
LotArea          0.388528
LotFrontage      0.332195
TotalBsmtSF      0.325883
WoodDeckSF       0.321450
HalfBath         0.291326
BsmtFullBath     0.230125
BsmtUnfSF        0.186572
BsmtFinSF1       0.185472
BedroomAbvGr     0.168213
2ndFlrSF         0.167849
ScreenPorch      0.093910
PoolArea         0.093700
3SsnPorch        0.047322
MoSold           0.046432
BsmtHalfBath    -0.016531
Id              -0.021917
YrSold          -0.028923
MSSubClass      -0.034217
LowQualFinSF    -0.044358
BsmtFinSF2      -0.048267
OverallCond     -0.053177
MiscVal         -0.063495
KitchenAbvGr    

# Handle categorial features via OHE

In [20]:
train_df = pd.get_dummies(train_df)

In [21]:
train_df.head(3)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1.0,4.110874,4.189655,9.042040,7.0,1.791759,7.602900,7.602900,5.283204,6.561031,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,3.044522,4.394449,9.169623,6.0,2.197225,7.589336,7.589336,0.000000,6.886532,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,4.110874,4.234107,9.328212,7.0,1.791759,7.601902,7.602401,5.093750,6.188264,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
train_df.shape

(1460, 305)

# Why the number of colomns increased?

In [23]:
for col in train_df.columns:
    print(col)

Id
MSSubClass
LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold
SalePrice
MSZoning_C (all)
MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Street_Grvl
Street_Pave
Alley_Grvl
Alley_None
Alley_Pave
LotShape_IR1
LotShape_IR2
LotShape_IR3
LotShape_Reg
LandContour_Bnk
LandContour_HLS
LandContour_Low
LandContour_Lvl
Utilities_AllPub
Utilities_NoSeWa
LotConfig_Corner
LotConfig_CulDSac
LotConfig_FR2
LotConfig_FR3
LotConfig_Inside
LandSlope_Gtl
LandSlope_Mod
LandSlope_Sev
Neighborhood_Blmngtn
Neighborhood_Blueste
Neighborhood_BrDale
Neighborhood_BrkSide
Neighborhood_ClearCr
Neighborhood_CollgCr
Neighborhood_Crawfor
Neighborhood_Edwards
Neighborhood_Gilbert
Neighborhood_

Because the categorial cols are now col per cat_value: PavedDrive --> PavedDrive_N, PavedDrive_P, PavedDrive_Y, each has only 1 or 0

Note that: if you do the same after filling NA with None, you get more colmns due to the _None cat.
    
Another way to do it is: LableEncoder. But it only works for ordinal not categorial vars, o.w. higher values means higher importance.

In [24]:
train_df.dtypes # No objects

Id                       float64
MSSubClass               float64
LotFrontage              float64
LotArea                  float64
OverallQual              float64
OverallCond              float64
YearBuilt                float64
YearRemodAdd             float64
MasVnrArea               float64
BsmtFinSF1               float64
BsmtFinSF2               float64
BsmtUnfSF                float64
TotalBsmtSF              float64
1stFlrSF                 float64
2ndFlrSF                 float64
LowQualFinSF             float64
GrLivArea                float64
BsmtFullBath             float64
BsmtHalfBath             float64
FullBath                 float64
HalfBath                 float64
BedroomAbvGr             float64
KitchenAbvGr             float64
TotRmsAbvGrd             float64
Fireplaces               float64
GarageYrBlt              float64
GarageCars               float64
GarageArea               float64
WoodDeckSF               float64
OpenPorchSF              float64
          

# Target variable

In [25]:
# Log transform the target for official scoring
train_df.SalePrice = np.log1p(train_df.SalePrice)
